# Simple baseline with Landsat and Bioclimatic Cubes + Sentinel images [0.31626]

Following the three provided baselies with different modalities, we have provide a multimodal approch based on "siamiese" network with multiple inputs and simple shared "decoder". The links for the separated baselines are as follows:

- [Baseline with Bioclimatic Cubes [0.25784]](https://www.kaggle.com/code/picekl/baseline-with-bioclimatic-cubes-0-25784)
- [Baseline with Landsat Cubes [0.26424]](https://www.kaggle.com/code/picekl/baseline-with-landsat-cubes-0-26424)
- [Baseline with Sentinel Images [0.23594]](https://www.kaggle.com/code/picekl/baseline-with-sentinel-images-0-23594)

**Considering the significant extent for enhancing performance of this baseline, we encourage you to experiment with various techniques, architectures, losses, etc.**

#### **Have Fun!**

# Data description

## Landsat time series

Satellite time series data includes over 20 years of Landsat satellite imagery extracted from [Ecodatacube](https://stac.ecodatacube.eu/).
The data was acquired through the Landsat satellite program and pre-processed by Ecodatacube to produce raster files scaled to the entire European continent and projected into a unique CRS.

Since the original rasters require a high amount of disk space, we extracted the data points from each spectral band corresponding to all PA and PO locations (i.e., GPS coordinates) and aggregated them in (i) CSV files and (ii) data cubes as tensor objects. Each data point corresponds to the mean value of Landsat's observations at the given location for three months before the given time; e.g., the value of a time series element under column 2012_4 will represent the mean value for that element from October 2012 to December 2012.

In this notebook, we will work with just the cubes. The cubes are structured as follows.
**Shape**: `(n_bands, n_quarters, n_years)` where:
- `n_bands` = 6 comprising [`red`, `green`, `blue`, `nir`, `swir1`, `swir2`]
- `n_quarters` = 4 
    - *Quarter 1*: December 2 of previous year until March 20 of current year (winter season proxy),
    - *Quarter 2*: March 21 until June 24 of current year (spring season proxy),
    - *Quarter 3*: June 25 until September 12 of current year (summer season proxy),
    - *Quarter 4*: September 13 until December 1 of current year (fall season proxy).
- `n_years` = 21 (ranging from 2000 to 2020)

The datacubes can simply be loaded as tensors using PyTorch with the following command :

```python
import torch
torch.load('path_to_file.pt')
```

**References:**
- *Traceability (lineage): This dataset is a seasonally aggregated and gapfilled version of the Landsat GLAD analysis-ready data product presented by Potapov et al., 2020 ( https://doi.org/10.3390/rs12030426 ).*
- *Scientific methodology: The Landsat GLAD ARD dataset was aggregated and harmonized using the eumap python package (available at https://eumap.readthedocs.io/en/latest/ ). The full process of gapfilling and harmonization is described in detail in Witjes et al., 2022 (in review, preprint available at https://doi.org/10.21203/rs.3.rs-561383/v3 ).*
- *Ecodatacube.eu: Analysis-ready open environmental data cube for Europe (https://doi.org/10.21203/rs.3.rs-2277090/v3).*


## Bioclimatic time series

The Bioclimatic Cubes are created from **four** monthly GeoTIFF CHELSA (https://chelsa-climate.org/timeseries/) time series climatic rasters with a resolution of 30 arc seconds, i.e. approximately 1km. The four variables are the precipitation (pr), maximum- (taxmax), minimum- (tasmin), and mean (tax) daily temperatures per month from January 2000 to June 2019. We provide the data in three forms: (i) raw rasters (GeoTiff images), (ii) CSV file with pre-extracted values for each location, i.e., surveyId, and (iii) data cubes as tensor object (.pt).

In this notebook, we will work with just the cubes. The cubes are structured as follows.
**Shape**: `(n_year, n_month, n_bio)` where:
- `n_year` = 19 (ranging from 2000 to 2018)
- `n_month` = 12 (ranging from January 01 to December 12)
- `n_bio` = 4 comprising [`pr` (precipitation), `tas` (mean daily air temperature), `tasmin`, `tasmax`]

The datacubes can simply be loaded as tensors using PyTorch with the following command :

```python
import torch
torch.load('path_to_file.pt')
```

**References:**
- *Karger, D.N., Conrad, O., Böhner, J., Kawohl, T., Kreft, H., Soria-Auza, R.W., Zimmermann, N.E., Linder, P., Kessler, M. (2017): Climatologies at high resolution for the Earth land surface areas. Scientific Data. 4 170122. https://doi.org/10.1038/sdata.2017.122*

- *Karger D.N., Conrad, O., Böhner, J., Kawohl, T., Kreft, H., Soria-Auza, R.W., Zimmermann, N.E, Linder, H.P., Kessler, M. Data from: Climatologies at high resolution for the earth’s land surface areas. Dryad Digital Repository. http://dx.doi.org/doi:10.5061/dryad.kd1d4*


## Sentinel Image Patches

The Sentinel Image data was acquired through the Sentinel2 satellite program and pre-processed by [Ecodatacube](https://stac.ecodatacube.eu/) to produce raster files scaled to the entire European continent and projected into a unique CRS. We filtered the data in order to pick patches from each spectral band corresponding to a location ((lon, lat) GPS coordinates) and a date matching that of our occurrences', and split them into JPEG files (RGB in 3-channels .jpeg files and NIR in single-channel .jpeg files) with a 128x128 resolution. The images were converted from sentinel uint15 to uint8 by clipping data pixel values over 10000 and applying a gamma correction of 2.5.

The data can simply be loaded using the following method:

```python
def construct_patch_path(output_path, survey_id):
    """Construct the patch file path based on survey_id as './CD/AB/XXXXABCD.jpeg'"""
    path = output_path
    for d in (str(survey_id)[-2:], str(survey_id)[-4:-2]):
        path = os.path.join(path, d)

    path = os.path.join(path, f"{survey_id}.jpeg")

    return path
```

**References:**
- *Traceability (lineage): The dataset was produced entirely by mosaicking and seasonally aggregating imagery from the Sentinel-2 Level-2A product (https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/product-types/level-2a)*
- *Ecodatacube.eu: Analysis-ready open environmental data cube for Europe (https://doi.org/10.21203/rs.3.rs-2277090/v3)*

In [1]:
import datetime
import json
import os
import shutil
import uuid
from typing import Literal

import numpy as np
import pandas as pd
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm.autonotebook import tqdm

RUN_TYPE = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "Localhost")
DEBUG = RUN_TYPE in (
    "Interactive",
    "Localhost",
)

DEBUG = False
experiment_name = None
data_dir = "/kaggle/input/geolifeclef-2024"
output_dir = "."

if experiment_name is None:
    experiment_name = (
        f"/{'trial' if DEBUG else 'run'}-"
        f"{datetime.datetime.now().strftime(r'%Y%m%d_%H%M%S')}-{uuid.uuid4().hex[:8]}/"
    )
if RUN_TYPE == "Localhost":
    data_dir = "./data/"
    output_dir = f"./runs/{experiment_name}/"
    os.makedirs(output_dir, exist_ok=True)

CONFIG = {
    "comment": "Baseline + Improved sentinel normalization +"
    " k-fold validation + impute with mean + 1/pos_weight",
    "seed": 151,
    "run_type": RUN_TYPE,
    "debug": DEBUG,
    "num_classes": 11255,
    "sentinel_transforms": transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.449, 0.485, 0.456, 0.406), std=(0.226, 0.229, 0.224, 0.225)
            ),
        ]
    ),
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "learning_rate": 3e-4,
    "num_epochs": 5 if DEBUG else 50,
    "train_val_split": 0.85,
    "batch_size": 128,
    "early_stopping_delay": 3,
}

num_classes = CONFIG["num_classes"]
seed = CONFIG["seed"]
device = CONFIG["device"]
tb_writer = SummaryWriter(log_dir=output_dir)
SENTINEL_TRANSFORMS = CONFIG["sentinel_transforms"]
if not os.path.exists(config_path := os.path.join(output_dir, "config.json")):
    with open(os.path.join(output_dir, "config.json"), "w") as fp:
        json.dump(
            CONFIG,
            fp,
            default=str,
        )
try:
    shutil.copyfile(__vsc_ipynb_file__, os.path.join(output_dir, "code.ipynb"))
except NameError:
    ...
output_dir

'./runs//trial-20240525_171152-80b9ccec//'

In [2]:
def set_seed(seed):
    # Set seed for Python's built-in random number generator
    torch.manual_seed(seed)
    # Set seed for numpy
    np.random.seed(seed)
    # Set seed for CUDA if available
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        # Set cuDNN's random number generator seed for deterministic behavior
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(seed)

## Prepare custom dataset loader

We have to slightly update the Dataset to provide the relevant data in the appropriate format.

In [3]:
def construct_patch_path(data_path, survey_id):
    """Construct the patch file path based on plot_id as './CD/AB/XXXXABCD.jpeg'."""
    survey_id = str(survey_id)

    return os.path.join(
        data_path, survey_id[-2:], survey_id[-4:-2], f"{survey_id}.jpeg"
    )


class TrainDataset(Dataset):
    def __init__(
        self,
        tab,
        bioclim_data_dir,
        landsat_data_dir,
        sentinel_data_dir,
        metadata,
        transform=None,
    ):
        self.tab = tab
        self.transform = transform
        self.sentinel_transform = transform = SENTINEL_TRANSFORMS

        self.bioclim_data_dir = bioclim_data_dir
        self.landsat_data_dir = landsat_data_dir
        self.sentinel_data_dir = sentinel_data_dir
        self.metadata = metadata
        self.metadata = self.metadata.dropna(subset="speciesId").reset_index(drop=True)
        self.metadata["speciesId"] = self.metadata["speciesId"].astype(int)
        self.label_dict = (
            self.metadata.groupby("surveyId")["speciesId"].apply(list).to_dict()
        )

        self.metadata = self.metadata.drop_duplicates(subset="surveyId").reset_index(
            drop=True
        )

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        survey_id = self.metadata.surveyId[idx]
        tab = torch.Tensor(
            self.tab[self.tab["surveyId"] == survey_id].iloc[:, 1:].values[0]
        )

        landsat_sample = torch.nan_to_num(
            torch.load(
                os.path.join(
                    self.landsat_data_dir,
                    f"GLC24-PA-train-landsat-time-series_{survey_id}_cube.pt",
                )
            )
        )
        bioclim_sample = torch.nan_to_num(
            torch.load(
                os.path.join(
                    self.bioclim_data_dir,
                    f"GLC24-PA-train-bioclimatic_monthly_{survey_id}_cube.pt",
                )
            )
        )

        rgb_sample = np.array(
            Image.open(construct_patch_path(self.sentinel_data_dir, survey_id))
        )
        nir_sample = np.array(
            Image.open(
                construct_patch_path(
                    self.sentinel_data_dir.replace("rgb", "nir").replace("RGB", "NIR"),
                    survey_id,
                )
            )
        )
        sentinel_sample = np.concatenate((nir_sample[..., None], rgb_sample), axis=2)

        species_ids = self.label_dict.get(
            survey_id, []
        )  # Get list of species IDs for the survey ID
        label = torch.zeros(num_classes)  # Initialize label tensor
        for species_id in species_ids:
            label_id = species_id
            label[label_id] = (
                1  # Set the corresponding class index to 1 for each species
            )

        if isinstance(landsat_sample, torch.Tensor):
            landsat_sample = landsat_sample.permute(
                1, 2, 0
            )  # Change tensor shape from (C, H, W) to (H, W, C)
            landsat_sample = landsat_sample.numpy()  # Convert tensor to numpy array

        if isinstance(bioclim_sample, torch.Tensor):
            bioclim_sample = bioclim_sample.permute(
                1, 2, 0
            )  # Change tensor shape from (C, H, W) to (H, W, C)
            bioclim_sample = bioclim_sample.numpy()  # Convert tensor to numpy array

        landsat_sample = self.transform(landsat_sample)
        bioclim_sample = self.transform(bioclim_sample)
        sentinel_sample = self.sentinel_transform(sentinel_sample)

        return tab, landsat_sample, bioclim_sample, sentinel_sample, label, survey_id


class TestDataset(TrainDataset):
    def __init__(
        self,
        tab,
        bioclim_data_dir,
        landsat_data_dir,
        sentinel_data_dir,
        metadata,
        transform=None,
    ):
        self.tab = tab
        self.transform = transform
        self.sentinel_transform = transform = SENTINEL_TRANSFORMS

        self.bioclim_data_dir = bioclim_data_dir
        self.landsat_data_dir = landsat_data_dir
        self.sentinel_data_dir = sentinel_data_dir
        self.metadata = metadata

    def __getitem__(self, idx):
        survey_id = self.metadata.surveyId[idx]
        tab = torch.Tensor(
            self.tab[self.tab["surveyId"] == survey_id].iloc[:, 1:].values[0]
        )

        landsat_sample = torch.nan_to_num(
            torch.load(
                os.path.join(
                    self.landsat_data_dir,
                    f"GLC24-PA-test-landsat_time_series_{survey_id}_cube.pt",
                )
            )
        )
        bioclim_sample = torch.nan_to_num(
            torch.load(
                os.path.join(
                    self.bioclim_data_dir,
                    f"GLC24-PA-test-bioclimatic_monthly_{survey_id}_cube.pt",
                )
            )
        )

        rgb_sample = np.array(
            Image.open(construct_patch_path(self.sentinel_data_dir, survey_id))
        )
        nir_sample = np.array(
            Image.open(
                construct_patch_path(
                    self.sentinel_data_dir.replace("rgb", "nir").replace("RGB", "NIR"),
                    survey_id,
                )
            )
        )
        sentinel_sample = np.concatenate((nir_sample[..., None], rgb_sample), axis=2)

        if isinstance(landsat_sample, torch.Tensor):
            landsat_sample = landsat_sample.permute(
                1, 2, 0
            )  # Change tensor shape from (C, H, W) to (H, W, C)
            landsat_sample = landsat_sample.numpy()  # Convert tensor to numpy array

        if isinstance(bioclim_sample, torch.Tensor):
            bioclim_sample = bioclim_sample.permute(
                1, 2, 0
            )  # Change tensor shape from (C, H, W) to (H, W, C)
            bioclim_sample = bioclim_sample.numpy()  # Convert tensor to numpy array

        landsat_sample = self.transform(landsat_sample)
        bioclim_sample = self.transform(bioclim_sample)
        sentinel_sample = self.sentinel_transform(sentinel_sample)

        return tab, landsat_sample, bioclim_sample, sentinel_sample, survey_id

### Load metadata and prepare data loaders

In [4]:
def create_tab(split: Literal["train", "test"]) -> pd.DataFrame:
    landcover = pd.read_csv(
        f"{data_dir}/EnvironmentalRasters/EnvironmentalRasters/LandCover/GLC24-PA-{split}-landcover.csv"
    )
    solidgrids = pd.read_csv(
        f"{data_dir}/EnvironmentalRasters/EnvironmentalRasters/SoilGrids/GLC24-PA-{split}-soilgrids.csv"
    )
    humanfootprint = pd.read_csv(
        f"{data_dir}/EnvironmentalRasters/EnvironmentalRasters/Human Footprint/"
        f"GLC24-PA-{split}-human_footprint.csv"
    )
    elevation = pd.read_csv(
        f"{data_dir}/EnvironmentalRasters/EnvironmentalRasters/Elevation/GLC24-PA-{split}-elevation.csv"
    )
    climate = pd.read_csv(
        f"{data_dir}/EnvironmentalRasters/EnvironmentalRasters/Climate"
        f"/Average 1981-2010/GLC24-PA-{split}-bioclimatic.csv"
    )
    tab = (
        climate.merge(elevation, on="surveyId")
        .merge(humanfootprint, on="surveyId")
        .merge(solidgrids, on="surveyId")
        .merge(landcover, on="surveyId")
    )
    tab = tab.replace(np.inf, np.nan).replace(-np.inf, np.nan)
    tab = pd.DataFrame(tab.values, tab.index, tab.columns, dtype=float)
    tab = tab.fillna(tab.mean())

    return tab


train_tab = create_tab(split="train")
test_tab = create_tab(split="test")
features = train_tab.columns[1:]

In [5]:
# Dataset and DataLoader
batch_size = CONFIG["batch_size"]
transform = transforms.Compose([transforms.ToTensor()])
train_val_split: float = CONFIG["train_val_split"]

train_metadata_path = f"{data_dir}/GLC24_PA_metadata_train.csv"


def dataset_loader():
    split_generator = torch.Generator().manual_seed(seed)
    train_landsat_data_path = (
        f"{data_dir}/TimeSeries-Cubes/TimeSeries-Cubes/"
        "GLC24-PA-train-landsat_time_series/"
    )
    train_bioclim_data_path = (
        f"{data_dir}/TimeSeries-Cubes/TimeSeries-Cubes/"
        "GLC24-PA-train-bioclimatic_monthly/"
    )
    train_sentinel_data_path = (
        f"{data_dir}/PA_Train_SatellitePatches_RGB/pa_train_patches_rgb/"
    )

    metadata = pd.read_csv(train_metadata_path)
    dataset = TrainDataset(
        train_tab,
        train_bioclim_data_path,
        train_landsat_data_path,
        train_sentinel_data_path,
        metadata,
        transform=transform,
    )

    # Load Test metadata
    test_landsat_data_path = (
        f"{data_dir}/TimeSeries-Cubes/TimeSeries-Cubes/"
        "GLC24-PA-test-landsat_time_series/"
    )
    test_bioclim_data_path = (
        f"{data_dir}/TimeSeries-Cubes/TimeSeries-Cubes/"
        "GLC24-PA-test-bioclimatic_monthly/"
    )
    test_sentinel_data_path = (
        f"{data_dir}/PA_Test_SatellitePatches_RGB/pa_test_patches_rgb/"
    )
    test_metadata_path = f"{data_dir}/GLC24_PA_metadata_test.csv"

    test_metadata = pd.read_csv(test_metadata_path)
    test_dataset = TestDataset(
        test_tab,
        test_bioclim_data_path,
        test_landsat_data_path,
        test_sentinel_data_path,
        test_metadata,
        transform=transform,
    )
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False, num_workers=4
    )

    class DatasetIterator:
        def __next__(self):
            training, validation = random_split(
                dataset,
                [
                    int(len(dataset) * train_val_split),
                    len(dataset) - int(len(dataset) * train_val_split),
                ],
                generator=split_generator,
            )
            train_loader = DataLoader(
                training, batch_size=batch_size, shuffle=True, num_workers=4
            )
            val_loader = DataLoader(
                validation, batch_size=batch_size, shuffle=False, num_workers=4
            )
            return (train_loader, val_loader), test_loader

    return DatasetIterator()

## Define and initialize a Multimodal Model

To process multiple inputs with different modalities and formats we use so-call siamiese approach where each modality is processed with different backbone (i.e., encoder). Data encoded into a 1d vector are concatenated and classified with a simple fully connected neural network. Short recap from previous notebooks.
- The Landsat cubes have a shape of [6,4,21] (BANDs, QUARTERs, and YEARs).
- The Bioclimatic cubes have a shape of [4,19,12] (RASTER-TYPE, YEAR, and MONTH)
- The Sentinel Image Patches have a shape od [128, 128, 4] (R, G, B, NIR)

In [6]:
class MultimodalEnsemble(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.tab_norm = nn.LayerNorm([len(features)])
        self.tab_model = nn.Sequential(
            nn.Linear(len(features), 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
        )

        self.landsat_norm = nn.LayerNorm([6, 4, 21])
        self.landsat_model = models.resnet18(weights=None)
        # Modify the first convolutional layer to accept 6 channels instead of 3
        self.landsat_model.conv1 = nn.Conv2d(
            6, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.landsat_model.maxpool = nn.Identity()

        self.bioclim_norm = nn.LayerNorm([4, 19, 12])
        self.bioclim_model = models.resnet18(weights=None)
        # Modify the first convolutional layer to accept 4 channels instead of 3
        self.bioclim_model.conv1 = nn.Conv2d(
            4, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bioclim_model.maxpool = nn.Identity()

        self.sentinel_model = models.swin_t(weights="IMAGENET1K_V1")
        self.sentinel_model.features[0][0] = nn.Conv2d(
            4,
            self.sentinel_model.features[0][0].out_channels,
            kernel_size=(4, 4),
            stride=(4, 4),
        )
        self.sentinel_model.head = nn.Identity()

        self.ln0 = nn.LayerNorm(64)
        self.ln1 = nn.LayerNorm(1000)
        self.ln2 = nn.LayerNorm(1000)
        self.ln3 = nn.LayerNorm(768)

        self.fc1 = nn.Linear(
            self.ln0.normalized_shape[0]
            + self.ln1.normalized_shape[0]
            + self.ln2.normalized_shape[0]
            + self.ln3.normalized_shape[0],
            1024,
        )
        self.fc2 = nn.Linear(1024, num_classes)

        self.dropout = nn.Dropout(p=0.15)

    def forward(self, t, x, y, z):
        t = self.tab_norm(t)
        t = self.tab_model(t)
        t = self.ln0(t)
        t = self.dropout(t)

        x = self.landsat_norm(x)
        x = self.landsat_model(x)
        x = self.ln1(x)
        x = self.dropout(x)

        y = self.bioclim_norm(y)
        y = self.bioclim_model(y)
        y = self.ln2(y)
        y = self.dropout(y)

        z = self.sentinel_model(z)
        z = self.ln3(z)
        z = self.dropout(z)

        txyz = torch.cat((t, x, y, z), dim=1)

        txyz = self.fc1(txyz).relu()
        txyz = self.dropout(txyz)

        out = self.fc2(txyz)
        return out


model = MultimodalEnsemble(num_classes).to(device)

## Training Loop

Nothing special, just a standard Pytorch training loop.

In [7]:
def calculate_f1_score(pred: np.ndarray, gt):
    list_f1 = []
    for p, g in zip(pred, gt, strict=False):
        sp = set(p)
        sg = set(g)
        TP = len(list(sp.intersection(sg)))
        FP = len(list(sp - sg))
        FN = len(list(sg - sp))
        f1 = TP / (TP + (FP + FN) / 2)
        list_f1.append(f1)
    return np.mean(list_f1)


def f1_score_and_top_k(survey_ids, predictions):
    train_pa = pd.read_csv(train_metadata_path)[["surveyId", "speciesId"]]
    train_pa[train_pa.columns] = train_pa[train_pa.columns].astype(int)
    gt = [
        train_pa[train_pa["surveyId"] == surveyId].speciesId.values.tolist()
        for surveyId in survey_ids
    ]
    pred_sorted = np.argsort(-np.array(predictions), axis=1)
    best_top_k = 1
    best_score = 0

    for k in range(1, 100):
        top_k = pred_sorted[:, :k].tolist()
        score = calculate_f1_score(top_k, gt)
        if score > best_score:
            best_score = score
            best_top_k = k
    return best_score, best_top_k

In [8]:
def calculate_pos_weights(dl: DataLoader) -> torch.Tensor:
    df = pd.read_csv(train_metadata_path)
    survey_ids = [df.loc[i, "surveyId"] for i in dl.dataset.indices]
    df = df[["surveyId", "speciesId"]][df.surveyId.isin(survey_ids)]
    df = df.drop_duplicates()

    pos_counts = df.groupby("speciesId")["surveyId"].nunique()

    output = torch.zeros(num_classes, dtype=torch.float32)
    num_samples = df.surveyId.nunique()
    weights = pos_counts.values
    weights = weights / (num_samples - weights)
    output[pos_counts.index.astype(int).to_numpy()] = torch.from_numpy(
        weights.astype(np.float32)
    )
    return output.to(device)

In [9]:
# Hyperparameters

learning_rate = CONFIG["learning_rate"]
num_epochs = CONFIG["num_epochs"]
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

best_val = None
early_stopping_delay = CONFIG["early_stopping_delay"]

dataset_iter = dataset_loader()

best_ago = 0

criterion = torch.nn.BCEWithLogitsLoss()
(train_loader, val_loader), test_loader = next(dataset_iter)
batch_pbar = tqdm(position=1, leave=True)
for epoch in tqdm(range(num_epochs), desc="Epoch", position=1):
    criterion.register_buffer("pos_weight", calculate_pos_weights(train_loader))

    # training
    total = 0
    total_loss = 0
    model.train()
    batch_pbar.reset(len(train_loader))
    batch_pbar.desc = f"Train {epoch}/{num_epochs}"
    for batch_idx, (data0, data1, data2, data3, targets, _) in enumerate(train_loader):
        batch_pbar.update()
        data0 = data0.to(device)
        data1 = data1.to(device)
        data2 = data2.to(device)
        data3 = data3.to(device)
        targets = targets.to(device)

        # Mixup
        if np.random.rand() < 0.4:
            lam = torch.tensor(np.random.beta(0.4, 0.4)).to(device)
            rand_index = torch.randperm(data0.size()[0]).to(device)
            mixed_data0 = lam * data0 + (1 - lam) * data0[rand_index]
            mixed_data1 = lam * data1 + (1 - lam) * data1[rand_index]
            mixed_data2 = lam * data2 + (1 - lam) * data2[rand_index]
            mixed_data3 = lam * data3 + (1 - lam) * data3[rand_index]
            targets_a, targets_b = targets, targets[rand_index]
            mixed_targets = lam * targets_a + (1 - lam) * targets_b
            outputs = model(mixed_data0, mixed_data1, mixed_data2, mixed_data3)
            loss = criterion(outputs, mixed_targets)
        else:
            outputs = model(data0, data1, data2, data3)
            loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total += data1.shape[0]
        total_loss += loss.item() * data1.shape[0]

        if DEBUG and batch_idx > 50:
            break

    total_loss /= total

    vtotal = 0
    vtotal_loss = 0
    model.eval()
    with torch.no_grad():
        batch_pbar.desc = f"Val {epoch}/{num_epochs}"
        batch_pbar.reset(len(val_loader))
        all_predictions = []
        all_surveyID = []
        for batch_idx, (data0, data1, data2, data3, targets, surveyID) in enumerate(
            val_loader
        ):
            batch_pbar.update()
            data0 = data0.to(device)
            data1 = data1.to(device)
            data2 = data2.to(device)
            data3 = data3.to(device)
            targets = targets.to(device)

            outputs = model(data0, data1, data2, data3)
            predictions = torch.sigmoid(outputs).cpu().numpy()

            all_predictions.extend(predictions)
            all_surveyID.extend(surveyID.numpy())

            loss = criterion(outputs, targets)

            vtotal += data1.shape[0]
            vtotal_loss += loss.item() * data1.shape[0]

            if DEBUG and batch_idx > 50:
                break
        f1_score, best_top_k = f1_score_and_top_k(all_surveyID, all_predictions)
        vtotal_loss /= vtotal
        tb_writer.add_scalar("train/loss", total_loss, epoch)
        tb_writer.add_scalar("val/loss", vtotal_loss, epoch)
        tb_writer.add_scalar("val/f1_score", f1_score, epoch)

    torch.save(model.eval().state_dict(), os.path.join(output_dir, "last.pth"))
    if best_val is None or vtotal_loss < best_val:
        best_val = vtotal_loss
        torch.save(model.eval().state_dict(), os.path.join(output_dir, "best.pth"))
        with open(os.path.join(output_dir, "thresholds.json"), "w") as fp:
            json.dump({"f1_score": f1_score, "best_top_k": best_top_k}, fp)
        best_ago = 0
        continue
    best_ago += 1
    if best_ago == early_stopping_delay:
        print(
            f"Stopping at epoch {epoch} as no improvement for {early_stopping_delay}"
            " epochs."
        )
        break
tb_writer.close()

0it [00:00, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

## Test Loop


Find top k for best model

In [10]:
with open(os.path.join(output_dir, "thresholds.json")) as fp:
    best_top_k = json.load(fp)["best_top_k"]

In [11]:
with torch.no_grad():
    all_predictions = []
    surveys = []
    top_k_indices = None
    for data0, data1, data2, data3, surveyID in tqdm(test_loader):
        data0 = data0.to(device)
        data1 = data1.to(device)
        data2 = data2.to(device)
        data3 = data3.to(device)

        outputs = model(data0, data1, data2, data3)
        predictions = torch.sigmoid(outputs).cpu().numpy()

        # Select top-k values as predictions
        top_k = np.argsort(-predictions, axis=1)[:, :best_top_k]
        if top_k_indices is None:
            top_k_indices = top_k
        else:
            top_k_indices = np.concatenate((top_k_indices, top_k), axis=0)

        surveys.extend(surveyID.cpu().numpy())

  0%|          | 0/37 [00:00<?, ?it/s]

## Save prediction file! 🎉🥳🙌🤗

In [12]:
data_concatenated = [" ".join(map(str, row)) for row in top_k_indices]

pd.DataFrame(
    {
        "surveyId": surveys,
        "predictions": data_concatenated,
    }
).to_csv(os.path.join(output_dir, "submission.csv"), index=False)